In [1]:
# import numpy as np
# import scipy.io as scio
# import pickle
# from default_Springer_HSMM_options import *
# from trainSpringerSegmentationAlgorithm import *
# from runSpringerSegmentationAlgorithm import *
# from plot_figs import *

# ## Load the default options:
# # These options control options such as the original sampling frequency of
# # the data, the sampling frequency for the derived features and whether the
# # mex code should be used for the Viterbi decoding:

# springer_options = default_Springer_HSMM_options(SpringerOptions)

# mat = scio.loadmat('example_data.mat')
# example_data = mat['example_data']

# # Abre o arquivo
# pkl_file = open('springerList.pkl', 'rb')
# B_matrix, pi_vector, total_obs_distribution = pickle.load(pkl_file)
# pkl_file.close()

In [2]:
# test_index = 663
# test_recordings = [example_data[0][0]['example_audio_data'][0][test_index]]
# test_annotations = [example_data[0][0]['example_annotations'][test_index][:]]

# ## Run the HMM on an unseen test recording:
# # And display the resulting segmentation
# numPCGs = len(test_recordings)

# PCGi = 0
# audio_data = test_recordings[PCGi]
# Fs = springer_options.audio_Fs

In [16]:
import numpy as np
import scipy.io as scio
import os
import librosa
import generateSegmentationData as gsd
import pathlib
import scipy.io as scio
import pickle
from trainSpringerSegmentationAlgorithm import *
from runSpringerSegmentationAlgorithm import *
from plot_figs import save_figs
from undoSegmentationData import *

## Load the default options:
# These options control options such as the original sampling frequency of
# the data, the sampling frequency for the derived features and whether the
# mex code should be used for the Viterbi decoding:

springer_options = default_Springer_HSMM_options(SpringerOptions)

# Carrega o modelo
filepath = '/home/linse/stevan/Qualirede/segmentador/springerList.pkl'
pkl_file = open(filepath, 'rb')
model = pickle.load(pkl_file)
pkl_file.close()

# Define os paths
inputRoot = '/media/linse/DADOS/stevan/datasets/heart-sound/training'
inputRoot = pathlib.Path(inputRoot)

outputRoot = '/media/linse/DADOS/stevan/datasets/heart-sound/python_segmentation/'
outputRoot = pathlib.Path(outputRoot)
outputRoot.mkdir(parents=True, exist_ok=True)

# segInputRoot = '/media/linse/DADOS/stevan/datasets/heart-sound/extra_data/hand_corrected_annotations_training/'
segInputRoot = '/media/linse/DADOS/stevan/datasets/heart-sound/extra_data/automated_Springer_annotations_training/'
segInputRoot = pathlib.Path(segInputRoot)

# Coloca as listas em ordem alfabética
folderList = sorted(list(inputRoot.glob('training*')))
segFolderList = sorted(list(segInputRoot.glob('training*')))

# 'training-a/', 'training-b/', 'training-c/', 'training-d/', 'training-e/', 'training-f/'
# for i in range(len(folderList)):
for i in range(1):
    fname = folderList[i].name
    
    inFolder = inputRoot / fname
    outFolder = outputRoot / fname
    outFolder.mkdir(parents=True, exist_ok=True)
    
    fname = fname+'-Aut'
    segInFolder = segInputRoot / fname
    
    fileList = sorted(list(inFolder.glob('*.wav')))
    segFileList = sorted(list(segInFolder.glob('*.mat')))
    
    fileList = fileList[44:45]

    for k in range(len(fileList)):
        recordName = inFolder / fileList[k]
        outFile = (outFolder / fileList[k].stem).with_suffix('.npz')
        fileName = segInFolder / segFileList[k]
        
        print(f'{folderList[i].name} | File {k+1} of {len(fileList)} | {fileList[k].stem}')
        
        # Load data and resample data
        PCG_resampled = librosa.load(recordName,sr=springer_options.audio_Fs)[0]

training-a | File 1 of 1 | a0045


In [17]:
## Load the default options:

springer_options = default_Springer_HSMM_options(SpringerOptions)
Fs = springer_options.audio_Fs

## Get model parameters

B_matrix, pi_vector, total_obs_distribution = model

In [18]:
import numpy as np
import librosa
from default_Springer_HSMM_options import *
from butterworth_filters import *
from schmidt_spike_removal import *
from normalise_signal import *
from Homomorphic_Envelope_with_Hilbert import *
from Hilbert_Envelope import *
from get_PSD_feature_Springer_HMM import *
from getDWT import *

audio_data = PCG_resampled

# def getSpringerPCGFeatures(audio_data, Fs, figures=False):
# function PCG_Features = getSpringerPCGFeatures(audio, Fs)
# Get the features used in the Springer segmentation algorithm.

springer_options = default_Springer_HSMM_options(SpringerOptions)

## Check to see if the Wavelet toolbox is available on the machine:
include_wavelet = springer_options.include_wavelet_feature 
featuresFs = springer_options.audio_segmentation_Fs # Downsampled feature sampling frequency

## 25-400Hz 4th order Butterworth band pass
audio_data = butterworth_low_pass_filter(audio_data,2,400,Fs)
audio_data = butterworth_high_pass_filter(audio_data,2,25,Fs)

In [302]:
import numpy as np
from statistics import median

original_signal = audio_data
fs = Fs
original_signal.shape

(35875,)

In [303]:
# def schmidt_spike_removal(original_signal, fs):
    
# Find the window size (500 ms)
windowsize = round(fs/2)

# Find any samples outside of a integer number of windows:
trailingsamples = len(original_signal) % windowsize

# Reshape the signal into a number of windows:
sampleframes = original_signal[:len(original_signal)-trailingsamples]

windowsize, trailingsamples, sampleframes.shape

(500, 375, (35500,))

In [304]:
sampleframes = sampleframes.reshape(windowsize,-1,order='F')
sampleframes.shape

(500, 71)

In [305]:
def get_MAAs(sampleframes):
    M = []
    for i in range(len(sampleframes[0,:])):
        M.append(max(abs(sampleframes[:,i])))
    MAAs = np.array(M)
    return MAAs

In [307]:
# Find the MAAs:
MAAs = get_MAAs(sampleframes)
MAAs, MAAs.shape

(array([0.05455746, 0.73185016, 0.06780181, 0.81726091, 0.09713932,
        0.74516327, 0.08630908, 0.69954506, 0.12447804, 0.77261722,
        0.22271207, 0.02881977, 0.74279339, 0.0216182 , 0.81761337,
        0.09786286, 0.81670202, 0.1087732 , 0.80511996, 0.0930815 ,
        0.81022931, 0.09154014, 0.79190463, 0.22212671, 0.03319415,
        0.79532517, 0.02610673, 0.80808411, 0.07082504, 0.77431265,
        0.11526874, 0.66513013, 0.0800697 , 0.84874737, 0.1013283 ,
        0.83440757, 0.1157187 , 0.83650264, 0.68542493, 0.02656417,
        0.75045641, 0.04563729, 0.79132721, 0.08801378, 0.77129449,
        0.06999517, 0.84241321, 0.16626033, 0.93131555, 0.09024964,
        0.90588139, 0.10629364, 0.03826258, 0.9304115 , 0.03236624,
        0.78606596, 0.10810866, 0.84546212, 0.09260469, 0.72531429,
        0.09129389, 0.85005561, 0.11465431, 0.82548012, 0.08355567,
        0.87218445, 0.28841796, 0.95476479, 0.05069621, 0.72143891,
        0.096065  ]), (71,))

In [308]:
median(MAAs)*3

0.668136204204411

In [310]:
(MAAs>median(MAAs)*3)*1

array([0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0])

In [311]:
np.nonzero(MAAs>median(MAAs)*3)

(array([ 1,  3,  5,  7,  9, 12, 14, 16, 18, 20, 22, 25, 27, 29, 33, 35, 37,
        38, 40, 42, 44, 46, 48, 50, 53, 55, 57, 59, 61, 63, 65, 67, 69]),)

In [312]:
MAAs[np.nonzero(MAAs>median(MAAs)*3)]

array([0.73185016, 0.81726091, 0.74516327, 0.69954506, 0.77261722,
       0.74279339, 0.81761337, 0.81670202, 0.80511996, 0.81022931,
       0.79190463, 0.79532517, 0.80808411, 0.77431265, 0.84874737,
       0.83440757, 0.83650264, 0.68542493, 0.75045641, 0.79132721,
       0.77129449, 0.84241321, 0.93131555, 0.90588139, 0.9304115 ,
       0.78606596, 0.84546212, 0.72531429, 0.85005561, 0.82548012,
       0.87218445, 0.95476479, 0.72143891])

In [193]:
# While there are still samples greater than 3* the median value of the
# MAAs, then remove those spikes:
emptyarray = np.nonzero(MAAs>median(MAAs)*3)[0]

In [283]:
not emptyarray.size==0

True

In [284]:
MAAs, max(MAAs), MAAs.shape

(array([0.05455746, 0.73185016, 0.06780181, 0.81726091, 0.09713932,
        0.74516327, 0.08630908, 0.69954506, 0.12447804, 0.77261722,
        0.22271207, 0.02881977, 0.74279339, 0.0216182 , 0.81761337,
        0.09786286, 0.81670202, 0.1087732 , 0.80511996, 0.0930815 ,
        0.81022931, 0.09154014, 0.79190463, 0.22212671, 0.03319415,
        0.79532517, 0.02610673, 0.80808411, 0.07082504, 0.77431265,
        0.11526874, 0.66513013, 0.0800697 , 0.84874737, 0.1013283 ,
        0.83440757, 0.1157187 , 0.83650264, 0.68542493, 0.02656417,
        0.75045641, 0.04563729, 0.79132721, 0.08801378, 0.77129449,
        0.06999517, 0.84241321, 0.16626033, 0.93131555, 0.09024964,
        0.90588139, 0.10629364, 0.03826258, 0.9304115 , 0.03236624,
        0.78606596, 0.10810866, 0.84546212, 0.09260469, 0.72531429,
        0.09129389, 0.85005561, 0.11465431, 0.82548012, 0.08355567,
        0.87218445, 0.28841796, 0.95476479, 0.05069621, 0.72143891,
        0.096065  ]), 0.9547647889300758, (71,))

In [285]:
np.argmax(MAAs, axis=None)

67

In [286]:
MAAs[np.argmax(MAAs, axis=None)]

0.9547647889300758

In [287]:
np.unravel_index(np.argmax(MAAs, axis=None), MAAs.shape)[0]

67

In [386]:
# Find the window with the max MAA:
val = max(MAAs)
# window_num = np.unravel_index(np.argmax(MAAs, axis=None), MAAs.shape)
# window_num = window_num[0]
window_num = np.argmax(MAAs)
val, window_num, window_num.size

(0.6670901500640112, 29, 1)

In [318]:
if(window_num.size>1):
    window_num = window_num[0]

In [319]:
sampleframes, sampleframes.shape

(array([[-0.00229255,  0.00047445, -0.00634267, ..., -0.03585618,
         -0.007799  ,  0.01133828],
        [-0.05430583, -0.00135892, -0.00720049, ..., -0.03960506,
         -0.00897305,  0.01718205],
        [-0.05455526, -0.00561921, -0.01073508, ..., -0.04193559,
         -0.00630515,  0.03538752],
        ...,
        [ 0.00176307, -0.01821329,  0.00266135, ...,  0.01101859,
         -0.02226684, -0.00164722],
        [ 0.00235558, -0.00898492,  0.00566291, ...,  0.00400381,
         -0.00060129, -0.00026829],
        [ 0.0006753 , -0.00386624,  0.00934696, ..., -0.00431992,
          0.01115824,  0.00046018]]), (500, 71))

In [320]:
abs(sampleframes[:,window_num]).shape

(500,)

In [321]:
val = max(abs(sampleframes[:,window_num]))
val

0.9547647889300758

In [322]:
spike_position = np.argmax(abs(sampleframes[:,window_num]))
spike_position

66

In [323]:
sampleframes[spike_position,window_num]

-0.9547647889300758

In [324]:
# Find the postion of the spike within that window:
val = max(abs(sampleframes[:,window_num]))
# spike_position = np.unravel_index(np.argmax(abs(sampleframes[:,window_num]), axis=None), abs(sampleframes[:,window_num]).shape)
# spike_position = spike_position[0]
spike_position = np.argmax(abs(sampleframes[:,window_num]))
val, spike_position

(0.9547647889300758, 66)

In [325]:
if(spike_position.size>1):
    spike_position = spike_position[0]

In [326]:
np.sign(sampleframes[:10,window_num])

array([-1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [327]:
np.diff(np.sign(sampleframes[:10,window_num]))

array([0., 0., 2., 0., 0., 0., 0., 0., 0.])

In [328]:
abs(np.diff(np.sign(sampleframes[:10,window_num])))

array([0., 0., 2., 0., 0., 0., 0., 0., 0.])

In [329]:
(abs(np.diff(np.sign(sampleframes[:10,window_num])))>1)*1

array([0, 0, 1, 0, 0, 0, 0, 0, 0])

In [330]:
le_zero_crossings = (abs(np.diff(np.sign(sampleframes[:,window_num])))>1)*1
le_zero_crossings, le_zero_crossings.shape

(array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [331]:
# Finding zero crossings (where there may not be actual 0 values, just a change from positive to negative):
zero_crossings = np.hstack((abs(np.diff(np.sign(sampleframes[:,window_num])))>1, 0))
zero_crossings,zero_crossings.shape

(array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [350]:
zero_crossings[:spike_position+1],zero_crossings[:spike_position+1].size

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1]), 67)

In [351]:
np.nonzero(zero_crossings[:spike_position+1])

(array([66]),)

In [352]:
not np.nonzero(zero_crossings[:spike_position+1])[0].size

False

In [353]:
nonzero = np.nonzero(zero_crossings[:spike_position+1])[0][-1]
nonzero

66

In [354]:
np.max((0,nonzero))

66

In [355]:
# Find the start of the spike, finding the last zero crossing before
# spike position. If that is empty, take the start of the window:
if (not np.nonzero(zero_crossings[:spike_position+1])[0].size):
    spike_start = 0
else:
    nonzero = np.nonzero(zero_crossings[:spike_position+1])[0][-1] #[-1] = MATLAB "last"
    spike_start = np.max((0,nonzero))
    
spike_start

66

In [356]:
zero_crossings[:spike_position]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [360]:
# Find the end of the spike, finding the first zero crossing after
# spike position. If that is empty, take the end of the window:
zero_crossings[:spike_position+1] = 0

In [362]:
zero_crossings[:spike_position+1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [363]:
np.nonzero(zero_crossings)

(array([ 73,  80,  85,  91, 100, 105, 109, 117, 125, 131, 136, 139, 145,
        159, 168, 181, 185, 188, 193, 197, 202, 209, 213, 217, 218, 222,
        229, 233, 238, 246, 251, 259, 264, 269, 271, 272, 278, 281, 283,
        285, 289, 296, 297, 302, 310, 315, 319, 323, 326, 328, 331, 337,
        340, 342, 352, 358, 365, 371, 374, 381, 393, 395, 402, 412, 430,
        448, 459, 466, 476, 484, 493]),)

In [364]:
not np.nonzero(zero_crossings)[0].size

False

In [365]:
nonzero = np.nonzero(zero_crossings)[0][0] #[0] = MATLAB "first"
nonzero

73

In [366]:
min(nonzero, windowsize)

73

In [368]:
if (not np.nonzero(zero_crossings)[0].size):
    spike_end = 0
else:
    nonzero = np.nonzero(zero_crossings)[0][0] #[0] = MATLAB "first"
    spike_end = min(nonzero, windowsize)
spike_end

73

In [369]:
sampleframes[spike_start:spike_end,window_num]

array([-9.54764789e-01,  1.00000000e-04,  1.00000000e-04,  1.00000000e-04,
        1.00000000e-04,  1.00000000e-04,  1.00000000e-04])

In [370]:
# Set to Zero
sampleframes[spike_start:spike_end,window_num] = 1e-6

In [371]:
sampleframes[spike_start:spike_end,window_num]

array([1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06, 1.e-06])

In [372]:
# Recaclulate MAAs
MAAs = get_MAAs(sampleframes)

In [373]:
MAAs, MAAs.shape, median(MAAs)*3

(array([0.05455746, 0.73185016, 0.06780181, 0.81726091, 0.09713932,
        0.74516327, 0.08630908, 0.69954506, 0.12447804, 0.77261722,
        0.22271207, 0.02881977, 0.74279339, 0.0216182 , 0.81761337,
        0.09786286, 0.81670202, 0.1087732 , 0.80511996, 0.0930815 ,
        0.81022931, 0.09154014, 0.79190463, 0.22212671, 0.03319415,
        0.79532517, 0.02610673, 0.80808411, 0.07082504, 0.77431265,
        0.11526874, 0.66513013, 0.0800697 , 0.84874737, 0.1013283 ,
        0.83440757, 0.1157187 , 0.83650264, 0.68542493, 0.02656417,
        0.75045641, 0.04563729, 0.79132721, 0.08801378, 0.77129449,
        0.06999517, 0.84241321, 0.16626033, 0.93131555, 0.09024964,
        0.90588139, 0.10629364, 0.03826258, 0.9304115 , 0.03236624,
        0.78606596, 0.10810866, 0.84546212, 0.09260469, 0.72531429,
        0.09129389, 0.85005561, 0.11465431, 0.82548012, 0.08355567,
        0.87218445, 0.28841796, 0.9479661 , 0.05069621, 0.72143891,
        0.096065  ]), (71,), 0.668136204204411)

In [379]:
old_val = val
old_window_num = window_num
old_val,old_window_num

(0.9479660990935009, 67)

In [380]:
# Find the window with the max MAA:
val = max(MAAs)
# window_num = np.unravel_index(np.argmax(MAAs, axis=None), MAAs.shape)
# window_num = window_num[0]
window_num = np.argmax(MAAs)
val, window_num, 

(0.9479660990935009, 67)

In [381]:
np.nonzero(MAAs>median(MAAs)*3)[0]

array([ 1,  3,  5,  7,  9, 12, 14, 16, 18, 20, 22, 25, 27, 29, 33, 35, 37,
       38, 40, 42, 44, 46, 48, 50, 53, 55, 57, 59, 61, 63, 65, 67, 69])

In [382]:
emptyarray = np.nonzero(MAAs>median(MAAs)*3)[0]

In [383]:
not emptyarray.size==0

True

### O loop amaldiçoado

In [385]:
while(not emptyarray.size==0):

    # Find the window with the max MAA:
    val = max(MAAs)
    window_num = np.argmax(MAAs)

    if(window_num.size>1):
        window_num = window_num[0]

    # Find the postion of the spike within that window:
    val = max(abs(sampleframes[:,window_num]))
    spike_position = np.argmax(abs(sampleframes[:,window_num]))

    if(spike_position.size>1):
        spike_position = spike_position[0]

    # Finding zero crossings (where there may not be actual 0 values, just a change from positive to negative):
    zero_crossings = np.hstack((abs(np.diff(np.sign(sampleframes[:,window_num])))>1, 0))

    # Find the start of the spike, finding the last zero crossing before
    # spike position. If that is empty, take the start of the window:
    if (not np.nonzero(zero_crossings[:spike_position+1])[0].size):
        spike_start = 0
    else:
        nonzero = np.nonzero(zero_crossings[:spike_position+1])[0][-1] #[-1] = MATLAB "last"
        spike_start = np.max((0,nonzero))

    # Find the end of the spike, finding the first zero crossing after
    # spike position. If that is empty, take the end of the window:
    zero_crossings[:spike_position+1] = 0
    if (not np.nonzero(zero_crossings)[0].size):
        spike_end = windowsize
    else:
        nonzero = np.nonzero(zero_crossings)[0][0] #[0] = MATLAB "first"
        spike_end = np.min((nonzero, windowsize))

    # Set to Zero
    sampleframes[spike_start:spike_end,window_num] = 0.0001

    # Recaclulate MAAs
    MAAs = get_MAAs(sampleframes)

    emptyarray = np.nonzero(MAAs>median(MAAs)*3)[0]

In [296]:
nonzero,windowsize

(66, 500)

In [300]:
np.min((nonzero, windowsize))

66

In [ ]:
despiked_signal = sampleframes.reshape(sampleframes.size,order='F')

# Add the trailing samples back to the signal:
despiked_signal = np.concatenate((despiked_signal, original_signal[despiked_signal.size:]))

# return despiked_signal

In [3]:
# Get PCG Features:

PCG_Features, featuresFs = getSpringerPCGFeatures(audio_data, Fs)

In [4]:
## Get PCG heart rate

heartRate, systolicTimeInterval = getHeartRateSchmidt(audio_data, Fs)

In [5]:
PCG_Features[:4,:]

array([[-0.25847775, -0.57497657, -0.34072   , -0.60936582],
       [ 0.15841158, -0.46887987, -0.29232661, -0.45480159],
       [-0.77324363, -0.53771186, -0.32784152, -0.10409826],
       [-0.60661867, -0.55654393, -0.28750247, -0.61659218]])

In [6]:
heartRate, systolicTimeInterval

(51.45797598627787, 0.389)

In [7]:
# _, _, qt = viterbiDecodePCG_Springer(PCG_Features, pi_vector, B_matrix, total_observation_distribution, heartRate, systolicTimeInterval, featuresFs)

import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from get_duration_distributions import *

def sigma(a):
    return 1/(1 + np.exp(-a))

def ind2sub(array_shape, ind):
    rows = (ind.astype('int') / array_shape[1])
    cols = (ind.astype('int') % array_shape[1]) # or numpy.mod(ind.astype('int'), array_shape[1])
    return (rows, cols)


In [8]:
observation_sequence, pi_vector, b_matrix, total_obs_distribution, heartrate, systolic_time, Fs = PCG_Features, pi_vector, B_matrix, total_obs_distribution, heartRate, systolicTimeInterval, featuresFs

## Preliminary
springer_options = default_Springer_HSMM_options(SpringerOptions)

T = len(observation_sequence)
N = 4 # Number of states

# Setting the maximum duration of a single state. This is set to an entire
# heart cycle:
max_duration_D = int(1*60/heartrate*Fs)
max_duration_D

58

In [9]:
# Initialising the variables that are needed to find the optimal state path along
# the observation sequence.
# delta_t(j), as defined on page 264 of Rabiner, is the best score (highest
# probability) along a single path, at time t, which accounts for the first
# t observations and ends in State s_j. In this case, the length of the
# matrix is extended by max_duration_D samples, in order to allow the use
# of the extended Viterbi algortithm:
delta = np.ones((T + max_duration_D-1, N))*(-np.inf)

# The argument that maximises the transition between states (this is
# basically the previous state that had the highest transition probability
# to the current state) is tracked using the psi variable.
psi = np.zeros((T + max_duration_D-1, N)).astype(int)

# An additional variable, that is not included on page 264 or Rabiner, is
# the state duration that maximises the delta variable. This is essential
# for the duration dependant HMM.
psi_duration = np.zeros((T + max_duration_D-1, N)).astype(int)

## Setting up observation probs
observation_probs = np.zeros((T,N))

In [10]:
for n in range(1):

    # MLR gives P(state|obs)
    # Therefore, need Bayes to get P(o|state)
    # P(o|state) = P(state|obs) * P(obs) / P(states)
    # Where p(obs) is derived from a MVN distribution from all
    # observations, and p(states) is taken from the pi_vector:
    w = b_matrix[0][n].ravel()
    X = np.hstack((np.ones(len(observation_sequence)).reshape(-1,1),observation_sequence))
    pihat = sigma(w @ X.T)


In [11]:
pihat.shape

(1700,)

In [12]:
pihat[:5].reshape(-1,1)

array([[0.33555502],
       [0.5394515 ],
       [0.13848172],
       [0.18146626],
       [0.16642614]])

In [13]:
for n in range(N):

    # MLR gives P(state|obs)
    # Therefore, need Bayes to get P(o|state)
    # P(o|state) = P(state|obs) * P(obs) / P(states)
    # Where p(obs) is derived from a MVN distribution from all
    # observations, and p(states) is taken from the pi_vector:
    w = b_matrix[0][n].ravel()
    X = np.hstack((np.ones(len(observation_sequence)).reshape(-1,1),observation_sequence))
    pihat = sigma(w @ X.T)

    for t in range(T):

        x = observation_sequence[t,:]
        mu = total_obs_distribution[0]
        cov = total_obs_distribution[1] # = sigma (MATLAB)
        mvn = multivariate_normal(mu,cov) #create a multivariate Gaussian object with specified mean and covariance matrix
        Po_correction = mvn.pdf(x) #evaluate the probability density at x
        observation_probs[t,n] = (pihat[t]*Po_correction)/pi_vector[0]

In [14]:
observation_probs.shape

(1700, 4)

In [15]:
observation_probs[:10,:]

array([[1.50559580e-01, 2.60667154e-01, 1.69191002e-01, 2.04472416e-01],
       [1.39834508e-01, 1.17681406e-01, 1.10757609e-01, 4.04782914e-02],
       [5.88074845e-02, 2.86233238e-01, 1.56197467e-01, 3.49877802e-01],
       [8.99057339e-02, 3.34505602e-01, 1.70937874e-01, 3.81023941e-01],
       [5.89872165e-02, 2.27850120e-01, 1.36387858e-01, 2.69029223e-01],
       [7.15089943e-02, 1.33101320e-01, 8.02054843e-02, 1.14450279e-01],
       [4.44027242e-02, 8.21119615e-03, 4.13252391e-02, 1.06086768e-03],
       [6.12838357e-04, 1.26304394e-05, 5.50259867e-04, 8.29586379e-08],
       [1.19612690e-07, 5.46737234e-10, 1.11252055e-07, 2.43497872e-13],
       [8.82985644e-15, 7.85593500e-17, 8.23790614e-15, 8.82969265e-20]])

In [16]:
## Setting up state duration probabilities, using Gaussian distributions:
d_distributions, max_S1, min_S1, max_S2, min_S2, max_systole, min_systole, max_diastole, min_diastole = get_duration_distributions(heartrate, systolic_time)

duration_probs = np.zeros((N,3*Fs))
duration_sum = np.zeros((N))
d_distributions, max_S1, min_S1, max_S2, min_S2, max_systole, min_systole, max_diastole, min_diastole

([[6, 1], [13, 1.5625], [5, 1], [34.15, 7.238790250000001]],
 9,
 3,
 8,
 2,
 19.75,
 6.25,
 42.2215,
 26.0785)

In [17]:
for state_j in range(N):
    for d in range(max_duration_D):

        if(state_j == 0):
            mvn = multivariate_normal(d_distributions[state_j][0], d_distributions[state_j][1])
            duration_probs[state_j,d] = mvn.pdf(d)

            if(d < min_S1 or d > max_S1):
                duration_probs[state_j,d] = np.finfo(float).tiny

        elif(state_j == 2):
            mvn = multivariate_normal(d_distributions[state_j][0], d_distributions[state_j][1])
            duration_probs[state_j,d] = mvn.pdf(d)

            if(d < min_S2 or d > max_S2):
                duration_probs[state_j,d] = np.finfo(float).tiny

        elif(state_j == 1):
            mvn = multivariate_normal(d_distributions[state_j][0], d_distributions[state_j][1])
            duration_probs[state_j,d] = mvn.pdf(d)

            if(d < min_systole or d > max_systole):
                duration_probs[state_j,d] = np.finfo(float).tiny

        elif(state_j == 3):
            mvn = multivariate_normal(d_distributions[state_j][0], d_distributions[state_j][1])
            duration_probs[state_j,d] = mvn.pdf(d)

            if(d < min_diastole or d > max_diastole):
                duration_probs[state_j,d] = np.finfo(float).tiny

    duration_sum[state_j] = sum(duration_probs[state_j,:])

In [18]:
duration_probs[:,:3]

array([[2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308],
       [2.22507386e-308, 2.22507386e-308, 4.43184841e-003],
       [2.22507386e-308, 2.22507386e-308, 2.22507386e-308]])

In [19]:
duration_probs.shape

(4, 150)

In [20]:
if(len(duration_probs)>3*Fs):
    duration_probs = duration_probs[:,:3*Fs]

## Perform the actual Viterbi Recursion:
qt = np.zeros(delta.size).astype(int)

In [21]:
## Initialisation Step

#Equation 32a and 69a, but leave out the probability of being in
#state i for only 1 sample, as the state could have started before time t = 0.

delta[0,:] = np.log(pi_vector) + np.log(observation_probs[0,:]) #first value is the probability of intially being in each state * probability of observation 1 coming from each state

In [22]:
delta.shape

(1757, 4)

In [23]:
delta

array([[-3.27969075, -2.73080532, -3.16302137, -2.97361656],
       [       -inf,        -inf,        -inf,        -inf],
       [       -inf,        -inf,        -inf,        -inf],
       ...,
       [       -inf,        -inf,        -inf,        -inf],
       [       -inf,        -inf,        -inf,        -inf],
       [       -inf,        -inf,        -inf,        -inf]])

In [24]:
#Equation 32b
psi[0,:] = -1
psi[:2,:]

array([[-1, -1, -1, -1],
       [ 0,  0,  0,  0]])

In [25]:
# The state duration probabilities are now used.
# Change the a_matrix to have zeros along the diagonal, therefore, only
# relying on the duration probabilities and observation probabilities to
# influence change in states:
# This would only be valid in sequences where the transition between states
# follows a distinct order.
a_matrix = np.array([[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0]])

### core Viterbi algorithm está DEMORADO

In [119]:
t = 1
j = 0
d = 0

start_t = t - d - 1
if(start_t<0):
    start_t = 0

if(start_t > T-2):
    start_t = T-2

end_t = t+1
if(t>T):
    end_t = T
    
start_t, end_t 

(0, 2)

In [120]:
epsilon = 1e-256
max_delta =    np.max(delta[start_t,:] + np.log(a_matrix[:,j]+epsilon))
max_index = np.argmax(delta[start_t,:] + np.log(a_matrix[:,j]+epsilon))
max_delta, max_index

(-2.9736165578398843, 3)

In [121]:
probs = np.prod(observation_probs[start_t:end_t,j])
probs

0.021053424824932974

In [122]:
if(probs == 0):
    probs = np.finfo(float).tiny
emission_probs = np.log(probs)
emission_probs

-3.860692032566122

In [123]:
if(emission_probs == 0 or np.isnan(emission_probs)):
    emission_probs = np.finfo(float).tiny
emission_probs

-3.860692032566122

In [124]:
delta_temp = max_delta + emission_probs + np.log((duration_probs[j,d]/duration_sum[j]))
delta_temp

-715.2304564453302

In [127]:
psi[t,j]

0

In [85]:
if(delta_temp > delta[t,j]):
    delta[t,j] = delta_temp
    psi[t,j] = max_index
    psi_duration[t,j] = d

(2.2250738585072014e-308, 0)

In [26]:
epsilon = 1e-256

## Run the core Viterbi algorithm


## Recursion

    ## The Extended Viterbi algorithm:

    # Equations 33a and 33b and 69a, b, c etc:
    # again, ommitting the p(d), as state could have started before t = 1

    # This implementation extends the standard implementation of the
    # duration-dependant Viterbi algorithm by allowing the durations to
    # extend beyond the start and end of the time series, thereby allowing
    # states to "start" and "stop" outside of the recorded signal. This
    # addresses the issue of partial states at the beginning and end of the
    # signal being labelled as the incorrect state. For instance, a
    # short-duration diastole at the beginning of a signal looks a lot like
    # systole, and can lead to labelling errors.

    # t spans input 2 to T + max_duration_D:

for t in range(1,T+max_duration_D-1):
    for j in range(N):
        for d in range(max_duration_D):

            # The start of the analysis window, which is the current time
            # step, minus d (the time horizon we are currently looking back),
            # plus 1. The analysis window can be seen to be starting one
            # step back each time the variable d is increased.
            # This is clamped to 1 if extending past the start of the
            # record, and T-1 is extending past the end of the record:
            start_t = t - d - 1
            if(start_t<0):
                start_t = 0

            if(start_t > T-2):
                start_t = T-2
                

            # The end of the analysis window, which is the current time
            # step, unless the time has gone past T, the end of the record, in
            # which case it is truncated to T. This allows the analysis
            # window to extend past the end of the record, so that the
            # timing durations of the states do not have to "end" at the end
            # of the record.
            end_t = t+1
            if(t>T):
                end_t = T

            # Find the max_delta and index of that from the previous step
            # and the transition to the current step:
            # This is the first half of the expression of equation 33a from
            # Rabiner:
            max_delta =    np.max(delta[start_t,:] + np.log(a_matrix[:,j]+epsilon))
            max_index = np.argmax(delta[start_t,:] + np.log(a_matrix[:,j]+epsilon))

            # Find the normalised probabilities of the observations over the
            # analysis window:
            probs = np.prod(observation_probs[start_t:end_t,j])

            # Find the normalised probabilities of the observations at only
            # the time point at the start of the time window:
            if(probs == 0):
                probs = np.finfo(float).tiny
            emission_probs = np.log(probs)

            # Keep a running total of the emmission probabilities as the
            # start point of the time window is moved back one step at a
            # time. This is the probability of seeing all the observations
            # in the analysis window in state j:
            if(emission_probs == 0 or np.isnan(emission_probs)):
                emission_probs = np.finfo(float).tiny

            # Find the total probability of transitioning from the last
            # state to this one, with the observations and being in the same
            # state for the analysis window. This is the duration-dependant
            # variation of equation 33a from Rabiner:
            #                 fprintf('log((duration_probs(j,d)./duration_sum(j))):%d\n',log((duration_probs(j,d)./duration_sum(j))));
            delta_temp = max_delta + emission_probs + np.log((duration_probs[j,d]/duration_sum[j]))

            # Unlike equation 33a from Rabiner, the maximum delta could come
            # from multiple d values, or from multiple size of the analysis
            # window. Therefore, only keep the maximum delta value over the
            # entire analysis window:
            # If this probability is greater than the last greatest,
            # update the delta matrix and the time duration variable:

            if(delta_temp > delta[t,j]):
                delta[t,j] = delta_temp
                psi[t,j] = max_index
                psi_duration[t,j] = d

In [32]:
temp_delta = delta[T:,:]
temp_delta.shape,temp_delta[:4,:]

((57, 4),
 array([[-5461.60722429, -5498.83351433, -5499.98473297, -5462.48968615],
        [-5461.10722429, -5495.8520516 , -5501.60098569, -5462.26174752],
        [-5460.91955165, -5492.9720516 , -5503.86118336, -5462.17195351],
        [-5460.48586912, -5490.25509126, -5506.36118336, -5462.22030413]]))

In [27]:
idx = np.argmax(temp_delta)
pos,_ = ind2sub(temp_delta.shape, idx)
pos = int(pos)
pos, idx

(3, 12)

In [28]:
pos = pos + T
pos

1703

In [29]:
np.ndarray.flatten(delta).shape

(7028,)

In [30]:
delta[pos,:]

array([-5460.48586912, -5490.25509126, -5506.36118336, -5462.22030413])

In [31]:
# 1)
state = np.argmax(delta[pos,:]) - 1 # -1 corrige para índices do Python
state

-1

In [33]:
# 2)
offset = pos + 1
preceding_state = psi[offset,state]
preceding_state

2

In [34]:
# 3)
# state_duration = psi_duration(offset, state);
onset = int(offset - psi_duration[offset,state])
onset

1668

In [35]:
# 4)
qt[onset:offset] = state
qt[onset:offset]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1])

In [36]:
# The state is then updated to the preceding state, found above, which must
# end when the last most likely state started in the observation sequence:
state = preceding_state

count = 0
# While the onset of the state is larger than the maximum duration
# specified:

In [37]:
while(onset > 1):

    #2)
    offset = onset - 1
    #     offset_array(offset,1) = inf;
    preceding_state = psi[offset,state]
    #     offset_array(offset,2) = preceding_state;


    #3)
    #     state_duration = psi_duration(offset, state);
    onset = offset - psi_duration[offset,state] + 1

    #4)
    #     offset_array(onset:offset,3) = state;

    if(onset<1):
        onset = 0

    qt[onset:offset] = state
    state = preceding_state
    count = count + 1

    if(count > 1000):
        break

qt = qt[:T]

In [44]:
qt[1000:1100]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 2, 2, 2, 2, 2, 0, 3, 3, 3, 3, 3])